<h1>Einfaches Word Embedding</h1>

Word Embeddings sind eine Form der Repräsentation von Wörtern in Zahlen, mit dem ein Model trainiert werden kann, um NLP Aufgaben zu lösen.

Mit Embeddings können Relationen gut abgebildet werden. <br>
Eine Methode Embeddings zu erstellen ist, durch Trainieren eines Models, das uns am Ende Embeddings liefert.
- Erstelle Fake-Problem, um eventuell Embeddings zu bekommen, die die Wörter gut repräsentiert

Als Einstieg in die Thematik sollen mittels einfachen Reviews Embeddings erstellt werden.  <br>
Das Prinzip ist einfach.

In [1]:
# // Content coming

In [2]:
# Imports.
## Achte auf Versionen, wenn import error ## 
import numpy as np
import pandas as pd
import gensim

import tensorflow as tf

from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding

In [6]:
# Mögliche Reviews:
reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

In [3]:
# Positiver oder negativer Review. 
y_truth = np.array([1,1,1,1,1,0,0,0,0,0])

Als Erstes muss das Vokabular erstellt werden. Jedes Wort bekommt eine Zahl, Relationen können damit nicht abgebildet werden.

Keras liefert eine Methode, mit dem das einfach umsetzbar ist.

In [7]:
# Input Text | n
# - Hat auch Filterfunktion. 
one_hot("eins zwei drei", 10)  # Vokabular Größe n, hier 10.

[8, 1, 1]

Damit bekommt jedes Wort eine Zahl.

Damit erstellen wir eine On-Hot-Encode Matrix. <br>
Diese Vektoren werden später für die Multiplikation benötigt. 

In [8]:
# Einfach als Liste.
ohe_review = [one_hot(wort, 10)  for wort in reviews ]
ohe_review

[[5, 9],
 [1, 1],
 [1, 7],
 [1, 8, 7],
 [4, 4, 9],
 [4, 9],
 [1, 4, 4],
 [1, 1],
 [1, 7],
 [2, 6]]

Das neurale Netz hat eine feste Größe, um eine verschiedene Anzahl von Wörtern (in einem Satz) zu verarbeiten, wird Padding angewendet, um den Rest mit Nullen zu füllen.

Dabei kann Keras uns auch wieder helfen, es schnell und sauber umzusetzen. 

In [9]:
# Keras übernimmt Padding.
# - Als Input: Review, Max. Länge des Satzes, Padding Typ. 
ohe_review_pad = tf.keras.preprocessing.sequence.pad_sequences(\
               ohe_review, maxlen=5, padding='post')
ohe_review_pad

array([[5, 9, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 7, 0, 0, 0],
       [1, 8, 7, 0, 0],
       [4, 4, 9, 0, 0],
       [4, 9, 0, 0, 0],
       [1, 4, 4, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 7, 0, 0, 0],
       [2, 6, 0, 0, 0]])

Der Rest wurde mit Nullen aufgefüllt. 

Für die Embeddings muss jetzt eine Vektorgröße festgelegt werden. 

In [10]:
# Vektorgröße:
vec_size = 6

# Model:
# - Siehe: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
model = tf.keras.Sequential([
    # Vokabulargröße | Vektorgröße  | Name
    tf.keras.layers.Embedding(10, vec_size, name="emb_1"),   # Embedding Layer
    tf.keras.layers.Flatten(),  # Flattern der Vektoren (10 x 6).
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(
    optimizer = 'adam',
    loss      = 'binary_crossentropy',
    metrics   = ['accuracy']
)

In [11]:
model.fit(ohe_review_pad, y_truth, epochs=30)

Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 586ms/step - accuracy: 0.5000 - loss: 0.6879
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5000 - loss: 0.6874
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5000 - loss: 0.6869
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5000 - loss: 0.6863
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5000 - loss: 0.6858
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5000 - loss: 0.6852
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5000 - loss: 0.6847
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5000 - loss: 0.6842
Epoch 9/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5000 - loss: 0.6836
Epoch 10/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6000 - loss: 0.6831
Epoch 11/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6000 - loss: 0.6825
Epoch 12/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6000 - loss: 0.6820


Die Embeddings, die wir brauchen, befinden sich in dem Model selber. <br>
Die Paramter des Models liefern und die Embeddings.


In [12]:
# Shape:
# - 10 x 6, da Vokabular 10, und Vektoren 6
model.get_layer('emb_1').get_weights()[0].shape

(10, 6)

In [13]:
model.get_layer('emb_1').get_weights()[0]

array([[ 0.02474366, -0.01315731,  0.05202236,  0.05083236,  0.01970943,
         0.0130841 ],
       [-0.01235311, -0.02102912,  0.0289436 , -0.0417017 ,  0.00227888,
        -0.03530249],
       [-0.04241379,  0.06800491, -0.07429703, -0.02135566, -0.06956505,
        -0.0467282 ],
       [-0.00316672,  0.02847575, -0.03855437,  0.03098381,  0.04959041,
         0.01081797],
       [-0.02940091,  0.04991698,  0.0132518 ,  0.05584835,  0.01868395,
         0.00743334],
       [ 0.06993849, -0.02619599,  0.00109671, -0.00600368,  0.06285763,
         0.07592013],
       [-0.02836825, -0.06181736,  0.00733179,  0.02542896,  0.06179117,
         0.0617546 ],
       [-0.00230336, -0.01995786, -0.07054552, -0.0401079 , -0.0379667 ,
        -0.00589087],
       [ 0.0043019 ,  0.05722755, -0.0754992 ,  0.01301988, -0.01062768,
        -0.01128125],
       [-0.01832882, -0.01759908,  0.00890784, -0.02409432, -0.06345172,
        -0.02548134]], dtype=float32)

In [14]:
emb = model.get_layer('emb_1').get_weights()[0]

Jetzt können wir uns die Embeddings für bestimmte Wörter anschauen.

In [15]:
# Nice:6, amazing: 4
emb[6]

array([-0.02836825, -0.06181736,  0.00733179,  0.02542896,  0.06179117,
        0.0617546 ], dtype=float32)

In [16]:
emb[4]

array([-0.02940091,  0.04991698,  0.0132518 ,  0.05584835,  0.01868395,
        0.00743334], dtype=float32)

In [17]:
# poor: 5
emb[5]

array([ 0.06993849, -0.02619599,  0.00109671, -0.00600368,  0.06285763,
        0.07592013], dtype=float32)

<h2>Word2Vec - Gensim</h2>

Word2Vec ist ein anderer Ansatz, um Wörter numerisch darzustellen.

Es  teilt sich in zwei Methoden auf:<br>
- CBOW: Continues Bag of Words.
- Skip Gram

Bei CBOW wird ein Text genommen und mit einem Fenster der Größe n-Worte über den Text gefahren. Dabei ist das Fake-Problem, die Wörter für die Lücken im Text vorherzusagen.

Skip-Gram macht genau das Gegenteil, es ist ein Wort gegeben, und das Model muss Wörter passend zum Kontext vorhersagen. 

Kontext: die Wörter die sich in der Umgebung der Lücke befinden.

Word2Vec ist eine Self-Supervised-Learning Methode, da es zu Beginn keine X und y Werte gibt, nur einen Text.  
- Aus dem Text: X und y Daten.

Für die Anwendung nutzen wir ein Dataset wo es um Produktbeschreibungen geht.

Dataset: reviews_Cell_Phones_and_Accessories_5
>https://www.kaggle.com/datasets/zainabhaddad/reviews-cell-phones-and-accessories-5 [Letzter Zurgiff: 18.06.2024]

In [19]:
review_df = pd.read_json('./data/datasets/reviewCellPhones.zip', lines=True, compression='zip')
review_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


Unser Ziel ist es Embeddings zu erstellen, deswegen interessieren wir uns nur für die Spalte "reviewText"

In [26]:
text_df = review_df['reviewText']
text_df.shape

(194439,)

Im nächsten Schritt bereiten wir die Daten vor, indem wir den Text filtern.
- Text sollte keine Zeichen haben wie !, //, usw.
- Der Text sollte nur Wörter haben, die man gut nutzen kann, Wörter wie "can't" müssen verändert oder entfernt werden.

Das Vorverarbeiten, etc., kann mit Funktionen und Libraries durchgeführt werden. <br>
Hier nutzen wir gensim. Siehe: https://pypi.org/project/gensim/ [Letzter Zurgiff: 18.06.2024]

In [34]:
# Es entfernt Wörter, die zu kurz oder zu lang sind. Der Text wird dabei in Lower-Case umgewandelt.
# - Mit Pandas kann die Funktion einfach auf alle Zellen angewendet werden. 
text_df = text_df.apply(gensim.utils.simple_preprocess)
text_df.head()  # Tokens

0    [they, look, good, and, stick, good, just, don...
1    [these, stickers, work, like, the, review, say...
2    [these, are, awesome, and, make, my, phone, lo...
3    [item, arrived, in, great, time, and, was, in,...
4    [awesome, stays, on, and, looks, great, can, b...
Name: reviewText, dtype: object

In [35]:
# Erstelle Gensim Model.
model = gensim.models.Word2Vec(
    window=10,   # Fenster 
    min_count=2, # Satz muss min. 2 Wörter haben. 
    workers=3    # CPU Worker
)
    

Dann brauchen wir wieder ein Vokabular. 
- Gensim hilft dabei auch.

In [36]:
model.build_vocab(text_df)

In [38]:
# total_examples: 194439
# - Oder durch model.corpus_count
model.train(text_df, total_examples=194439, epochs=6)

(73808871, 100642770)

Danach kann das Model mit save(...) gespeichert werden, um es später zu nutzen. 

Eine weitere nette Sache die man mit Gensim machen kann, ist sich die ähnlichen Wörter ausgeben zu lassen. 
- Dabei weisen die Embeddings Ähnlichkeiten auf.

In [39]:
model.wv.most_similar('good')

[('decent', 0.8170683979988098),
 ('great', 0.7875612378120422),
 ('nice', 0.7136105895042419),
 ('fantastic', 0.7091127038002014),
 ('excellent', 0.6294456720352173),
 ('superb', 0.6257697939872742),
 ('exceptional', 0.6123889684677124),
 ('awesome', 0.5947822332382202),
 ('outstanding', 0.5888533592224121),
 ('terrific', 0.5848925709724426)]

Beispiel mit einem anderen Dataset: meta_Sports_and_Outdoors

> https://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ [Letzer Zugriff: 18.06.2024]

In [40]:
del model, text_df, review_df

In [57]:
text_df =pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz", lines=True)
text_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,5,Woks very good,1390694400,"01 26, 2014"
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,5,Works as well as the factory tool,1328140800,"02 2, 2012"
2,AB2W04NI4OEAD,1881509818,J. Fernald,"[2, 2]",If you don't have a 3/32 punch or would like t...,4,"It's a punch, that's all.",1330387200,"02 28, 2012"
3,A148SVSWKTJKU6,1881509818,"Jusitn A. Watts ""Maverick9614""","[0, 0]",This works no better than any 3/32 punch you w...,4,It's a punch with a Glock logo.,1328400000,"02 5, 2012"
4,AAAWJ6LW9WMOO,1881509818,Material Man,"[0, 0]",I purchased this thinking maybe I need a speci...,4,"Ok,tool does what a regular punch does.",1366675200,"04 23, 2013"


Auch hier ändert sich das Vorgehen nicht.

In [58]:
# Wähle Text aus.
review_text = text_df['reviewText'] 
# Filtere Text
review_text = review_text.apply(gensim.utils.simple_preprocess)
review_text.head()

0    [this, came, in, on, time, and, am, veru, happ...
1    [had, factory, glock, tool, that, was, using, ...
2    [if, you, don, have, punch, or, would, like, t...
3    [this, works, no, better, than, any, punch, yo...
4    [purchased, this, thinking, maybe, need, speci...
Name: reviewText, dtype: object

In [59]:
# Erstelle Gensim Model.
model = gensim.models.Word2Vec(
    window=8,    # Fenster 
    min_count=2, # Satz muss min. 2 Wörter haben. 
    workers=4    # CPU Worker
)

In [60]:
# Erstelle Vokabular.
model.build_vocab(review_text)

In [61]:
model.corpus_count

296337

In [62]:
# Trainiere.
model.train(review_text, total_examples=296337, epochs=6)

(109606952, 145795842)

In [64]:
model.wv.most_similar('good')

[('decent', 0.8598130941390991),
 ('great', 0.7914591431617737),
 ('nice', 0.7311326265335083),
 ('fantastic', 0.7098391056060791),
 ('terrific', 0.6529357433319092),
 ('excellent', 0.6398428678512573),
 ('reasonable', 0.6331174373626709),
 ('superb', 0.630240797996521),
 ('awesome', 0.627801775932312),
 ('wonderful', 0.6105560064315796)]

In [65]:
model.wv.similarity(w1="slow",   w2='steady')

0.3634112